In [5]:
%pip install Beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [7]:
!playwright install

In [5]:
# Import libraries
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import Constants

In [8]:
# Get the html
async def get_html(url, selector, sleep = 5, retries = 3):
    html = None
    
    for i in range(1, retries + 1):
        
        # Create a sleep period, so we don't get banned from sites for scraping
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                
                # Print scraped data
                print(await page.title())
                html = await page.inner_html(selector)
                
        except PlaywrightTimeout:
            # If tomeout, show me
            print(f"Timeout error on {url}")
            continue
        else:
            break
        
    return html
                

In [9]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]

    standing_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    
    for url in standing_pages:
        
        # save scraped data in directory
        save_path = os.path.join(Constants.STANDINGS_DIR, url.split("/")[-1])
        
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [10]:
for season in Constants.SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com


CancelledError: 

In [11]:
standings_files = os.listdir(Constants.STANDINGS_DIR)

In [14]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(Constants.SCORES_DIR, url.split("/")[-1])
        
        if os.path.exists(save_path):
            continue
            
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [16]:
standings_files = [s for s in standings_files if ".html" in s]

for f in standings_files:
    filepath = os.path.join(Constants.STANDINGS_DIR, f)
        
    await scrape_game(filepath)

Wizards vs Raptors, October 20, 2021 | Basketball-Reference.com
Cavaliers vs Grizzlies, October 20, 2021 | Basketball-Reference.com
Rockets vs Timberwolves, October 20, 2021 | Basketball-Reference.com
76ers vs Pelicans, October 20, 2021 | Basketball-Reference.com
Magic vs Spurs, October 20, 2021 | Basketball-Reference.com
Thunder vs Jazz, October 20, 2021 | Basketball-Reference.com
Kings vs Trail Blazers, October 20, 2021 | Basketball-Reference.com
Nuggets vs Suns, October 20, 2021 | Basketball-Reference.com
Mavericks vs Hawks, October 21, 2021 | Basketball-Reference.com
Bucks vs Heat, October 21, 2021 | Basketball-Reference.com
Clippers vs Warriors, October 21, 2021 | Basketball-Reference.com
Knicks vs Magic, October 22, 2021 | Basketball-Reference.com
Pacers vs Wizards, October 22, 2021 | Basketball-Reference.com
Hornets vs Cavaliers, October 22, 2021 | Basketball-Reference.com
Raptors vs Celtics, October 22, 2021 | Basketball-Reference.com
Nets vs 76ers, October 22, 2021 | Basketbal